In [1]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.linalg import block_diag
from collections import defaultdict
import random
import csv

In [2]:
# Define the parameters

# Path to the CSV file containing the cost data
data_path = "./cost_10_07.csv"

# Column name in the CSV file that represents the part
part_col = "Part"

# Column name in the CSV file that represents the departure/arrival site
dep_arr_site_col = "Departure/Arrival site"

# Column name in the CSV file that represents the arrival/departure site
arr_dep_site_col =  "Arrival/Departure site"

# Column name in the CSV file that represents the cost
cost_col = "Cost"

# List of tuples representing the part and its corresponding site
# Each tuple is in the format (part, site)
PBS = [(2, 1), (3, 1), (4, 1), (5, 2) , (6, 2), (7, 2), (8, 3), (9, 4), (10, 4)]

# List of tuples representing the subparts
# Each tuple is in the format (subpart1, subpart2)
#subpart = [(2,3),(2,4),(3,4),(5,6),(5,7),(6,7),(9,10)]

# Penalty parameters for the optimization problem
lambda_1 = 1
lambda_2 = 1
lambda_3 = 1

In [3]:
# Read data from the CSV file
data = pd.read_csv(data_path)
# Store it in a pandas dataframe
data_df = pd.DataFrame(data)

In [4]:
# Define the function to create the cost matrix DataFrame
def cost_matrix_df(data_df, part_col, dep_arr_site_col, arr_dep_site_col, cost_col):

    # Define the range of parts and sites
    # The range of parts starts from 2 and goes up to the maximum part number
    # The range of sites starts from 1 and goes up to the maximum site number
    part_range = range(1, data_df[part_col].max() + 1)
    site_range = range(1, max(data_df[dep_arr_site_col].max(), data_df[arr_dep_site_col].max()) + 1)

    pivot_df = (
        pd.DataFrame(
            index=pd.MultiIndex.from_product(
                [part_range, site_range, site_range], 
                names=['Part', 'Departure_Site', 'Arrival_Site']
            ) # Create a pivot DataFrame with a multi-index of parts, departure sites, and arrival sites
        )
        .reset_index() # Rest index of the dataframe
        .merge(data_df.assign(Part_Column=data_df[part_col]), how='left'
               , left_on=['Part', 'Departure_Site', 'Arrival_Site'] # Merge the pivot DataFrame with the original DataFrame on the part, departure site, and arrival site columns
               , right_on=[part_col, dep_arr_site_col, arr_dep_site_col])
        .assign(Cost=lambda df: df[cost_col].fillna(0)) # Fill the missing cost values with 0
        .pivot_table(index=['Part', 'Departure_Site']
                     , columns=['Part_Column', 'Arrival_Site']
                     , values='Cost'
                     , fill_value=0) # Pivot the DataFrame to create a cost matrix with the part and departure site as the index and the part and arrival site as the columns
        .pipe(lambda df: df.reorder_levels(['Part_Column', 'Arrival_Site'], axis=1))    # Reorder the levels of the columns
        .pipe(lambda df: df.reindex(columns=pd.MultiIndex.from_product([part_range, site_range]
                                                                       , names=df.columns.names)
                                    , fill_value=0)) # Reindex the columns and the index to include all combinations of parts and sites
        .pipe(lambda df: df.reindex(index=pd.MultiIndex.from_product([part_range, site_range], names=df.index.names), fill_value=0))
        .pipe(lambda df: df + df.T) # Make the DataFrame symmetric by adding its transpose to itself
        .pipe(lambda df: df.where(np.eye(df.shape[0], dtype=bool), df / 2)) # Halve the off-diagonal elements and keep the diagonal the same
    )

    # Return the pivot DataFrame
    return pivot_df

In [5]:
def subparts(PBS):
    # Initialize the subpart set
    subpart_set = set()

    # Loop over all tuples in PBS
    for r, s in PBS:
        # Loop over all other tuples in PBS
        for r2, s2 in PBS:
            # Check if r and r2 are sub-parts of a common part at the next level up in the PBS
            if s == s2 and r < r2:
                subpart_set.add((r, r2))

    return subpart_set

In [6]:
def cost_function_qubo(PBS):
    pivot_df = cost_matrix_df(data_df, part_col, dep_arr_site_col, arr_dep_site_col, cost_col)
    site_range = pivot_df.index.get_level_values('Departure_Site').unique()

    # Get the maximum part number from PBS
    max_part = max(r for r, _ in PBS)

    cost_function = {(f'x_{r},{i}', f'x_{r},{j}'): pivot_df.loc[(r, i), (r, j)] 
            for r in range(1, max_part + 1) for i in site_range for j in site_range if (r, i) in pivot_df.index and (r, j) in pivot_df.index}

    return cost_function

In [7]:
def constraint_C1(PBS):
    # Initialize the constraint dictionary
    constraint_C1 = {}
    
    max_sites = data_df['Arrival/Departure site'].max()
    # Get the maximum part number from PBS
    max_part = max(r for r, _ in PBS)

    # Define the range of sites
    site_range = range(1, max_sites + 1)  # Sites go from 1 to num_sites

    # Loop over all parts and sites to build the QUBO terms
    for part in range(1, max_part + 1):
        for i in site_range:
            for j in site_range:
                if i == j:
                    # Diagonal terms: -2 * x_{part i}^2
                    constraint_C1[(f'x_{part},{i}', f'x_{part},{i}')] = -2
                else:
                    # Off-diagonal terms: x_{part i} * x_{part j}
                    constraint_C1[(f'x_{part},{i}', f'x_{part},{j}')] = 1

    return constraint_C1

In [8]:
def constraint_C2(PBS):
    # Define the range of sites
    num_sites = data_df['Arrival/Departure site'].max()
    # Get the maximum part number from PBS
    max_part = max(r for r, _ in PBS)

    # Initialize the constraint dictionary
    constraint_C2 = {}
    
    # Loop over all parts part in the PBS and sites i to build the QUBO terms
    for part in range(1, max_part + 1):
        for i in range(1, num_sites + 1):
            # Since r and s are the same, we only need to consider the terms x_{ri} x_{ri}
            # Add a penalty term for assigning part r to the same site as both origin and destination
            constraint_C2[(f'x_{part},{i}', f'x_{part},{i}')] = 1  # The penalty weight is set to 1

    return constraint_C2

In [9]:
def constraint_C3(PBS, subparts):
    # Define the range of sites
    num_sites = data_df['Arrival/Departure site'].max()

    # Initialize the constraint dictionary
    constraint_C3 = {}

    # Loop over all tuples in subparts and sites to build the QUBO terms
    for r, s in subparts:
        # Check if r and s are sub-parts of a common part at the next level up in the PBS
        if any((r, t) in PBS and (s, t) in PBS for t in range(1, num_sites + 1)):
            for i in range(1, num_sites + 1):
                # Add a penalty term for assigning part r to the same site as both origin and destination
                for j in range(i+1, num_sites + 1):
                    constraint_C3[(f'x_{r},{i}', f'x_{s},{j}')] = 1  # The penalty weight is set to 1

    return constraint_C3

In [10]:
def final_qubo(C, C1, C2, C3, lambda_1, lambda_2, lambda_3):
    # Initialize the final QUBO dictionary
    final_qubo = {}

    # Add the terms from each QUBO dictionary, multiplied by the corresponding lambda value
    for qubo_dict, lambda_value in zip([C, C1, C2, C3], [1, lambda_1, lambda_2, lambda_3]):
        for key, value in qubo_dict.items():
            if key in final_qubo:
                final_qubo[key] += lambda_value * value
            else:
                final_qubo[key] = lambda_value * value

    # Return the final QUBO dictionary
    return final_qubo

In [11]:
def qubo_matrix_df(Q):
    Q_df = (
        pd.Series(Q)  # Convert the QUBO dictionary into a pandas Series
        .reset_index()  # Reset the index
        .pivot(
            index='level_0', columns='level_1', values=0)  # Pivot the DataFrame to create a matrix
        .fillna(0)  # Fill NaN values with 0
        .rename_axis(index=None, columns=None)  # Remove the index and column names
    )
    
    Q_df = Q_df.reindex(sorted(Q_df.columns, key=lambda x: [int(num) for num in x[2:].split(',')]), axis=1)
    Q_df = Q_df.reindex(sorted(Q_df.index, key=lambda x: [int(num) for num in x[2:].split(',')]), axis=0)

    # Add the lower triangular values to the upper triangular ones, leaving the diagonal unchanged
    Q_np = Q_df.to_numpy()
    Q_df = pd.DataFrame(Q_np + Q_np.T - np.diag(np.diag(Q_np)), columns=Q_df.columns, index=Q_df.index)

    # Convert the DataFrame to a numpy array, make it upper triangular, and then convert it back to a DataFrame
    Q_df = pd.DataFrame(np.triu(Q_df.values), columns=Q_df.columns, index=Q_df.index)

    return Q_df

In [12]:
# Build the list of subparts
subpart = list(subparts(PBS))

In [13]:
# C is the QUBO matrix for the cost function.
# The cost_function_qubo function takes PBS as an argument and returns
# the QUBO matrix.
C = cost_function_qubo(PBS)

# C1 is the QUBO matrix for the first penalty term.
# The penalty_C1_qubo function takes PBS as an argument and returns
# the first penalty term.
C1 = constraint_C1(PBS)

# C2 is the QUBO matrix for the second penalty term.
# The penalty_C2_qubo function takes PBS as an argument and returns
# the second penalty term.
C2 = constraint_C2(PBS)

# C3 is the QUBO matrix for the third penalty term.
# The penalty_C3_qubo function takes PBS and subpart as arguments and returns
# the third penalty term.
C3 = constraint_C3(PBS, subpart)

In [14]:
# Create final QUBO matrix
qubo_dict = final_qubo(C, C1, C2, C3, lambda_1, lambda_2, lambda_3)

In [15]:
# Convert the QUBO dictionary to a DataFrame
Q = qubo_matrix_df(qubo_dict)

# Set the display options
pd.set_option('display.max_columns', None)  # No limit on columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping to next line
pd.set_option('display.max_rows', None)  # No limit on rows

# Print the DataFrame
print(Q)

        x_1,1  x_1,2  x_1,3  x_1,4  x_1,5  x_1,6  x_1,7  x_2,1  x_2,2  x_2,3  x_2,4  x_2,5  x_2,6  x_2,7  x_3,1  x_3,2  x_3,3  x_3,4  x_3,5  x_3,6  x_3,7  x_4,1  x_4,2  x_4,3  x_4,4  x_4,5  x_4,6  x_4,7  x_5,1  x_5,2  x_5,3  x_5,4  x_5,5  x_5,6  x_5,7  x_6,1  x_6,2  x_6,3  x_6,4  x_6,5  x_6,6  x_6,7  x_7,1  x_7,2  x_7,3  x_7,4  x_7,5  x_7,6  x_7,7  x_8,1  x_8,2  x_8,3  x_8,4  x_8,5  x_8,6  x_8,7  x_9,1  x_9,2  x_9,3  x_9,4  x_9,5  x_9,6  x_9,7  x_10,1  x_10,2  x_10,3  x_10,4  x_10,5  x_10,6  x_10,7
x_1,1    -1.0    2.0    2.0    2.0    2.0    2.0    2.0    0.0   0.00   0.00   0.00   0.00   0.00   0.00    0.0   0.00   0.00   0.00   0.00   0.00   0.00    0.0   0.00   0.00   0.00   0.00   0.00   0.00    0.0   0.00   0.00   0.00   0.00   0.00   0.00    0.0   0.00   0.00   0.00   0.00   0.00   0.00    0.0    0.0   0.00   0.00   0.00   0.00   0.00    0.0   0.00   0.00   0.00   0.00   0.00   0.00    0.0   0.00   0.00   0.00   0.00   0.00   0.00     0.0     0.0    0.00    0.00    0.00    0.00 